In [454]:
# Here is a little matrix class with some random stuff from linear algrebra
# I know I should use numpy but I wanted to implement it myself (even if it is less efficient)

In [475]:
# A few basic functions
def dot_product(v1,v2):
    total = 0
    for x, y in zip(v1,v2):
        total += x*y
    return total

def add_vectors(v1,v2):
    return [x+y for x, y in zip(v1,v2)]

def subtract_vectors(v1,v2):
    return [x-y for x, y in zip(v1,v2)]

def scalar_multiply(c, v):
    return [c*x for x in v]

def vector_length(v):
    return sqrt(sum([(a**2) for a in v]))

# my basic matrix multiplication algorithm
def matrix_multiplication(A,B):
    if len(A[0]) != len(B):
        raise Exception("Cannot multiply matrices, # columns of A must equal # rows of B")
    
    result = [([0] * len(B[0])) for _ in range(0,len(A))]
    
    for Arow_ix, Arow in enumerate(A):
        for Brow_ix, Brow in enumerate(B): 
            result[Arow_ix][Brow_ix] = dot_product(Arow, Brow)
    
    return result

def transpose(matrix):
        columns = []
        for column in range(0,len(matrix[0])):
            columns.append([row[column] for row in matrix])
        return columns

In [476]:
print("Dot product of <1,2,3> and <4,5,6> = {0}".format(dot_product([1,2,3],[4,5,6])))
print("Sum of vectors <1,2,3> and <4,5,6> = {0}".format(add_vectors([1,2,3],[4,5,6])))
print("Difference of vectors <1,2,3> and <4,5,6> = {0}".format(subtract_vectors([1,2,3],[4,5,6])))
print("Scalar multiple 4 times vector <1,2,3> = {0}".format(scalar_multiply(4,[1,2,3])))
print("Length of vector <1,2,3> = {0}".format(vector_length([1,2,3])))
print("Matrix A=[[1,2],[3,4]] x B=[[5,6],[7,8]] = {0}".format(matrix_multiplication([[1,2],[3,4]],[[5,6],[7,8]])))
print("Transpose of matrix A=[[1,2],[3,4]] = {0}".format(transpose([[1,2],[3,4]])))

Dot product of <1,2,3> and <4,5,6> = 32
Sum of vectors <1,2,3> and <4,5,6> = [5, 7, 9]
Difference of vectors <1,2,3> and <4,5,6> = [-3, -3, -3]
Scalar multiple 4 times vector <1,2,3> = [4, 8, 12]
Length of vector <1,2,3> = 3.7416573867739413
Matrix A=[[1,2],[3,4]] x B=[[5,6],[7,8]] = [[17, 23], [39, 53]]
Transpose of matrix A=[[1,2],[3,4]] = [[1, 3], [2, 4]]


In [484]:
from fractions import Fraction
from math import sqrt

# Gram-Schmidt: from a basis create find an orthogonal basis
def gram_schmidt(basis, fraction=True):
    # v1 = x1 always
    basis = [[Fraction(x) for x in v] for v in basis]
    basis = transpose(basis)
    orthogonal_basis = [basis[0]]
    
    # vp = xp - (dot(xp,vp-1)/dot(vp-1,vp-1))(vp-1) -...- (dot(xp,v1)/dot(v1,v1))(v1)
    for vector in basis[1:]:
        for ortho_vector in orthogonal_basis:
            temp = scalar_multiply((dot_product(vector, ortho_vector) /  dot_product(ortho_vector, ortho_vector)), 
                                   ortho_vector)
            
            vector = subtract_vectors(vector, temp)
        orthogonal_basis.append(vector)
    
    return orthogonal_basis

def orthonormal_basis(basis, fraction=True):
    orthogonal_basis = gram_schmidt(basis, fraction=fraction)
    
    orthonormal_basis = []
    for column in orthogonal_basis:
        length = vector_length(column)
        orthonormal_basis.append([(x/length) for x in column])
        
    return orthonormal_basis

In [485]:
A = [[1,0,0],
     [1,1,0],
     [1,1,1],
     [1,1,1]]

print("Orthogonal basis from basis A:")
for x in gram_schmidt(A):
    print(x)

print()

print("Orthonormal basis from basis A:")
for x in orthonormal_basis(A):
    print(x)

Orthogonal basis from basis A:
[Fraction(1, 1), Fraction(1, 1), Fraction(1, 1), Fraction(1, 1)]
[Fraction(-3, 4), Fraction(1, 4), Fraction(1, 4), Fraction(1, 4)]
[Fraction(0, 1), Fraction(-2, 3), Fraction(1, 3), Fraction(1, 3)]

Orthonormal basis from basis A:
[0.5, 0.5, 0.5, 0.5]
[-0.8660254037844387, 0.2886751345948129, 0.2886751345948129, 0.2886751345948129]
[0.0, -0.816496580927726, 0.408248290463863, 0.408248290463863]


In [505]:
from copy import deepcopy
from fractions import Fraction
from math import sqrt

class Matrix:
    """Matrix class for random things you can do to a matrix, may add bases stuff to this
    because you can essentially treat them as matrices"""
    
    def __init__(self, matrix):
        "Create mxn matrix"
        try:
            n = len(matrix[0])
        except TypeError:
            raise Exception("Matrix should be constructed from lists")
        
        for column in matrix:
            if len(column) != n:
                raise Exception("Rows must be the same length")
        
        self.matrix = matrix
    
    def dot_product(self,v1,v2):
        total = 0
        for x, y in zip(v1,v2):
            total += x*y
        return total
    
    def add_vectors(self,v1,v2):
        return [x+y for x, y in zip(v1,v2)]

    def subtract_vectors(self,v1,v2):
        return [x-y for x, y in zip(v1,v2)]

    def scalar_multiply(self,c, v):
        return [c*x for x in v]

    def vector_length(self,v):
        return sqrt(sum([(a**2) for a in v]))

    # my basic matrix multiplication algorithm
    def matrix_multiplication(self,A,B):
        if len(A[0]) != len(B):
            raise Exception("Cannot multiply matrices, # columns of A must equal # rows of B")

        result = [([0] * len(B[0])) for _ in range(0,len(A))]

        for Arow_ix, Arow in enumerate(A):
            for Brow_ix, Brow in enumerate(B): 
                result[Arow_ix][Brow_ix] = self.dot_product(Arow, Brow)

        return result
    
    def size(self):
        return (len(self.matrix), len(self.matrix[0]))
    
    def isSquare(self):
        "Check if the matrix is square"
        return len(self.matrix) == len(self.matrix[0])
    
    def transpose(self):
        columns = []
        for column in range(0,len(self.matrix[0])):
            columns.append([row[column] for row in self.matrix])
        return columns
    
    def invertible(self):
        if not self.isSquare():
            return False
        
        test = [0]*len(self.matrix[0])
        for row in self.matrix:
            if row == test:
                return False
        return True
            
    def echelonForm(self, matrix="self"):
        "Use row reduction to get echelon form of matrix"
        A = self.matrix if matrix == "self" else matrix
        row = 0
        column = 0

        while row < len(A) and column < len(A[0]):
            if A[row][column] != 0:
                A[row] = [x/A[row][column] for x in A[row]]

                for i in range(row+1,len(A)):
                        if A[i][column] != 0:
                            A[i] = [x + y for x, y in zip(A[i], [x*-A[i][column] for x in A[column]])]
            column+=1
            row+=1
            
        # find pivot with 1 in first position TODO?
        return A
    
    # fix this TODO
    def reducedEchelonForm(self, matrix="self"):
        A = self.echelonForm()
        
        for r_ix, row in enumerate(reversed(A)):
            try:
                pivot_column = row.index(1)
                pivot_row = len(A) - (r_ix+1)
                # inner loop
                for ri, r in enumerate(A):
                    if r[pivot_column] != 0 and ri != pivot_row:
                        A[ri] = [x + y for x, y in zip(A[ri], [x*-A[ri][pivot_column] for x in A[pivot_row]])]
            except:
                pass
        return A
    
    def __reducedEchelonForm(self,A):
        for r_ix, row in enumerate(reversed(A)):
            try:
                pivot_column = row.index(1)
                pivot_row = len(A) - (r_ix+1)
                # inner loop
                for ri, r in enumerate(A):
                    if r[pivot_column] != 0 and ri != pivot_row:
                        A[ri] = [x + y for x, y in zip(A[ri], [x*-A[ri][pivot_column] for x in A[pivot_row]])]
            except:
                pass
        return A
    
    
    def __determinant_helper(self, matrix):
        if len(matrix) == 2:
            return (matrix[0][0]*matrix[1][1]) - (matrix[1][0]*matrix[0][1])

        # alternatively, find row or column with fewest non-zero values TODO

        total = 0

        # make the minor
        for ix, value in enumerate(matrix[0]):
            if value != 0:
                temp_mat = []
                for i, row in enumerate(matrix):
                    if i != 0:
                        temp_row = []
                        for j, val in enumerate(row):
                            if j != ix:
                                temp_row.append(val)
                        temp_mat.append(temp_row)

                # -1^(j+1)a1jdetA1j
                total += ((-1)**(ix)) * value * determinant(temp_mat)
        return total
    
    def determinant(self):
        if self.isSquare():
            return self.__determinant_helper(self.matrix)
    
    # fix memory stuff...
    def inverse(self):
        if not self.invertible():
            raise Exception("Matrix not invertible")
        
        identity_matrix = []
        for x in range(0,len(self.matrix)):
            temp = []
            for y in range(0,len(self.matrix)):
                if x == y:
                    temp.append(1)
                else:
                    temp.append(0)
            identity_matrix.append(temp)
                    
        A = deepcopy(self.matrix)
        for ix, (a_row, i_row) in enumerate(zip(A,identity_matrix)):
            A[ix] = a_row + i_row
        
        # got a little frustrated with some functions. just made a new matrix instead.
        temp = Matrix(A)
        inverse = []
        n = len(self.matrix[0])
        
        for row in temp.reducedEchelonForm():
            inv_row = []
            for ix in range(n,(2*n)):
                inv_row.append(row[ix])
            inverse.append(inv_row)
             
        return inverse
    
    # Gram-Schmidt: from a basis create find an orthogonal basis
    def gram_schmidt(self, basis, fraction=True):
        # v1 = x1 always
        basis = [[Fraction(x) for x in v] for v in basis]
        orthogonal_basis = [basis[0]]

        # vp = xp - (dot(xp,vp-1)/dot(vp-1,vp-1))(vp-1) -...- (dot(xp,v1)/dot(v1,v1))(v1)
        for vector in basis[1:]:
            for ortho_vector in orthogonal_basis:
                temp = self.scalar_multiply((dot_product(vector, ortho_vector) /  dot_product(ortho_vector, ortho_vector)), 
                                       ortho_vector)

                vector = self.subtract_vectors(vector, temp)
            orthogonal_basis.append(vector)

        return orthogonal_basis

    def orthonormal_basis(self, basis, fraction=True):
        orthogonal_basis = gram_schmidt(basis, fraction=fraction)
        orthonormal_basis = []
        for column in orthogonal_basis:
            length = vector_length(column)
            orthonormal_basis.append([(x/length) for x in column])
        
        return orthonormal_basis
    
    # QR factorization
    def eigenValues(self):
        #seriously need transpose function to call (static)
        A = self.orthonormal_basis(self.transpose())
        #R = Qt A
        R = matrix_multiplication(A, self.matrix)
        return [R[x][x] for x in range(len(R))]
    
    def eigenVectors(self):
        pass
    
    def leastSquares(self, b):
        pass

In [560]:
def mat_mul2(A,B):
    if len(A[0]) != len(B):
        raise Exception("Cannot multiply matrices, # columns of A must equal # rows of B")
        
    result = []
    for col_ix in range(0, len(B[0])):
        temp = []
        for ix, row in enumerate(A):
            temp.append(dot_product(row,[B[x][col_ix] for x in range(0,len(B))]))
        result.append(temp)
    return result


# AT Ax = AT b
# x = ((AT A)^(-1)) ((AT b))

A = [[4,0],
     [0,2],
     [1,1]]

b = [[2],[0],[11]]


def leastSquares(A,b):
    AT_A = Matrix(mat_mul2(transpose(A),A))
    AT_b = mat_mul2(transpose(A),b)    
    
    return mat_mul2(AT_b,AT_A.inverse())

for x in leastSquares(A,b):
    print(x)

[1.0]
[2.0]


In [547]:
# feel free to ignore this stuff

In [500]:
# A bunch of matrices you can try out
A = [[1,0,0],
     [1,1,0],
     [1,1,1],
     [1,1,1]]

B = [[1,2,3,4],
     [5,6,7,8],
     [9,10,11,12]]

C = [[-.7,.3,.3],
     [.4,-.9,.5],
     [.3,.6,-.8]]

D = [[12,24,-4],
     [6,-2,-2],
     [-9,5,3],
     [-3,1,1]]

E = [[1,0,3],
     [-1,1,-6],
     [2,4,1]]

In [501]:
M = Matrix(E)

In [502]:
M.isSquare()

True

In [503]:
M.determinant()

7

In [490]:
M.eigenValues()

[3.162277660168379, 1.3784048752090223, 1.6059101370939326]

In [492]:
for x in M.echelonForm():
    print(x)

[1.0, 0.0, 3.0]
[0.0, 1.0, -3.0]
[0.0, 0.0, 1.0]


In [494]:
for x in M.reducedEchelonForm():
    print(x)

[1.0, 0.0, 0.0]
[0.0, 1.0, 0.0]
[0.0, 0.0, 1.0]


In [504]:
M.inverse()

[[3.571428571428571, 1.7142857142857142, -0.42857142857142855],
 [-1.5714285714285712, -0.7142857142857142, 0.42857142857142855],
 [-0.8571428571428571, -0.5714285714285714, 0.14285714285714285]]